# 🤝 **Faithfulness**

   - **What It Measures**: Here, you're checking if the answers stick to the facts provided in the context. It's all about staying true to the source.

   - **Scoring**: Also on a scale of 0 to 1. Higher values mean your answer is a faithful representation of the context.

   - **Example**:

     - ❓ Question: What is the significance of the Apollo 11 mission?

     - 📑 Context: Apollo 11 was the first manned mission to land on the Moon in 1969.

     - ⬆️ High faithfulness answer: Apollo 11 is significant as it was the first mission to bring humans to the Moon.

     - ⬇️ Low faithfulness answer: Apollo 11 was significant for its study of Mars.